# RAGAS Evaluation Metrics

## Install required packages.

Please offer any feedback if there are additional required packages so that I can make any updates.

In [ ]:
%pip install ragas
%pip install langchain-together

## Construct the evaluation function.

In [ ]:
def ragas(dataset_for_eval):
    # Load the dataset.
    import json
    from datasets import Dataset
    with open(dataset_for_eval, 'r', encoding='utf-8') as file:
        data_raw = json.load(file)
    dataset = Dataset.from_dict(data_raw)

    # Deploy the model from LangChain Together AI.
    from langchain_together import Together
    from langchain_together.embeddings import TogetherEmbeddings
    import os
    os.environ["TOGETHER_API_KEY"] = "ceeef42d7bed8f09c9ca2b2a5e81d55b5389398566cd20698d12238c79a68abb"
    together_key = "ceeef42d7bed8f09c9ca2b2a5e81d55b5389398566cd20698d12238c79a68abb"
    embeddings = TogetherEmbeddings(model="togethercomputer/m2-bert-80M-8k-retrieval")
    together_completion = Together(
        model="NousResearch/Nous-Hermes-2-Mixtral-8x7B-SFT",
        temperature=0.7,
        max_tokens=4000,
        top_k=1,
        together_api_key=together_key
    )

    # Conduct RAGAS through four main metrics.
    from ragas import evaluate
    from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
    score = evaluate(dataset, metrics=[faithfulness, answer_relevancy, context_precision, context_recall], llm=together_completion, embeddings=embeddings, raise_exceptions=False)
    score_df = score.to_pandas()
    # print(score_df)
    score_df.to_csv(dataset_for_eval, index=False)
    return

## Run the evaluation metrics.

Note that the `dataset_for_eval.json` should be replaced with the dataset generated from `new_rag.py`.

In [ ]:
ragas("dataset_for_eval.json")